# widgets.image_cleaner

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

In [ ]:
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageCleaner, ImageDownloader, download_google_images
from fastai.gen_doc.nbdoc import show_doc

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.save('stage-1')

We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)

In [ ]:
db = (ImageItemList.from_folder(path)
                   .no_split()
                   .label_from_folder()
                   .databunch())

In [ ]:
learn = create_cnn(db, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

In [ ]:
show_doc(DatasetFormatter)

<h2 id="DatasetFormatter"><code>class</code> <code>DatasetFormatter</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L14" class="source_link">[source]</a></h2>

> <code>DatasetFormatter</code>()

Returns a dataset with the appropriate format and file indices to be displayed.  

The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

In [ ]:
show_doc(DatasetFormatter.from_similars)

<h4 id="DatasetFormatter.from_similars"><code>from_similars</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L35" class="source_link">[source]</a></h4>

> <code>from_similars</code>(<b>`learn`</b>, <b>`layer_ls`</b>:`list`=<b><i>`[0, 7, 2]`</i></b>, <b>`kwargs`</b>)

Gets the indices for the most similar images.  

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.widgets.image_cleaner import * 

In [ ]:
show_doc(DatasetFormatter.from_toplosses)

<h4 id="DatasetFormatter.from_toplosses"><code>from_toplosses</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L16" class="source_link">[source]</a></h4>

> <code>from_toplosses</code>(<b>`learn`</b>, <b>`n_imgs`</b>=<b><i>`None`</i></b>, <b>`kwargs`</b>)

Gets indices with top losses.  

In [ ]:
show_doc(ImageCleaner)

<h2 id="ImageCleaner"><code>class</code> <code>ImageCleaner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L96" class="source_link">[source]</a></h2>

> <code>ImageCleaner</code>(<b>`dataset`</b>, <b>`fns_idxs`</b>, <b>`path`</b>, <b>`batch_size`</b>:`int`=<b><i>`5`</i></b>, <b>`duplicates`</b>=<b><i>`False`</i></b>)

Displays images for relabeling or deletion and saves changes in `path` as 'cleaned.csv'.  

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system. To use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) we must first use `DatasetFormatter().from_toplosses` to get the suggested indices for misclassified images.

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

In [ ]:
ImageCleaner(ds, idxs, path)

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) does not change anything on disk (neither labels or existence of images). Instead, it creates a 'cleaned.csv' file in your data path from which you need to load your new databunch for the files to changes to be applied. 

In [ ]:
df = pd.read_csv(path/'cleaned.csv', header='infer')

In [ ]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageItemList.from_df(df, path)
                   .no_split()
                   .label_from_df()
                   .databunch(bs=64))

In [ ]:
learn = create_cnn(db, models.resnet18, metrics=error_rate)
learn = learn.load('stage-1')

You can then use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) again to find duplicates in the dataset. To do this, you can specify `duplicates=True` while calling ImageCleaner after getting the indices and dataset from `.from_similars`. Note that if you are using a layer's output which has dimensions [n_batches, n_features, 1, 1] then you don't need any pooling (this is the case with the last layer). The suggested use of `.from_similars()` with resnets is using the last layer and no pooling, like in the following cell.

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn, layer_ls=[0,7,1], pool=None)

Getting activations...


Computing similarities...


In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

In [ ]:
show_doc(ImageDownloader)

<h2 id="ImageDownloader"><code>class</code> <code>ImageDownloader</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L11" class="source_link">[source]</a></h2>

> <code>ImageDownloader</code>(<b>`path`</b>:`PathOrStr`=<b><i>`'data'`</i></b>)

Displays a widget that allows searching and downloading images from google images search in a Jupyter Notebook or Lab. 

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) widget gives you a way to quickly bootstrap your image dataset without leaving the notebook. It searches and downloads images that match the search criteria and resolution / quality requirements and stores them on your filesystem within the provided `path`.

Images for each search query (or label) are stored in a separate folder within `path`. For example, if you pupulate `tiger` with a `path` setup to `./data`, you'll get a folder `./data/tiger/` with the tiger images in it.

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) will automatically clean up and verify the downloaded images with [`verify_images()`](/vision.data.html#verify_images) after downloading them.

In [ ]:
path = Path('./image_downloader_data')
ImageDownloader(path)

After populating images with [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader), you can get a an [`ImageDataBunch`](/vision.data.html#ImageDataBunch) by calling `ImageDataBunch.from_folder(path, size=size)`, or using the data block API.

In [ ]:
path.ls()

In [ ]:
src = (ImageItemList.from_folder(path)
       .random_split_by_pct()
       .label_from_folder()
       .transform(get_transforms(), size=224))
db  = src.databunch(bs=16)

In [ ]:
learn = create_cnn(db, models.resnet34, metrics=[accuracy])

In [ ]:
learn.fit_one_cycle(3)

#### Downloading more than a hundred images

To fetch more than a hundred images, [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) uses `selenium` and `chromedriver` to scroll through the Google Images search results page and scrape image URLs. They're not required as dependencies by default. If you don't have them installed on your system, the widget will show you an error message.

To install `selenium`, just `pip install selenium` in your fastai environment.

**On a mac**, you can install `chromedriver` with `brew cask install chromedriver`.

**On Ubuntu**
Take a look at the latest Chromedriver version available, then something like:

```
wget https://chromedriver.storage.googleapis.com/2.45/chromedriver_linux64.zip
unzip chromedriver_linux64.zip
```

#### Downloading images in python scripts outside Jupyter notebooks

In [ ]:
path = Path('image_downloader_data')

In [ ]:
download_google_images(path, 'aussie shepherd', size='>1024*768', n_images=150)

In [ ]:
show_doc(download_google_images)

<h4 id="download_google_images"><code>download_google_images</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L93" class="source_link">[source]</a></h4>

> <code>download_google_images</code>(<b>`path`</b>:`PathOrStr`, <b>`search_term`</b>:`str`, <b>`size`</b>:`str`=<b><i>`'>400*300'`</i></b>, <b>`n_images`</b>:`int`=<b><i>`10`</i></b>, <b>`format`</b>:`str`=<b><i>`'jpg'`</i></b>, <b>`max_workers`</b>:`int`=<b><i>`8`</i></b>, <b>`timeout`</b>:`int`=<b><i>`4`</i></b>) → `FilePathList`

Search for `n_images` images on Google, matching `search_term` and `size` requirements, and download them into `path`/`search_term` directory.

Automatically [`verify_images`](/vision.data.html#verify_images) and return the image file names list.

Uses `max_workers` threads to download and verify images. 

Note that downloading under 100 images doesn't require any dependencies other than fastai itself, however downloading more than a hundred images [uses `selenium` and `chromedriver`](/widgets.image_cleaner.html#Downloading-more-than-a-hundred-images).

`size` can be one of:

```
'>400*300'
'>640*480'
'>800*600'
'>1024*768'
'>2MP'
'>4MP'
'>6MP'
'>8MP'
'>10MP'
'>12MP'
'>15MP'
'>20MP'
'>40MP'
'>70MP'
```

## Methods

## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(ImageCleaner.make_dropdown_widget)

<h4 id="ImageCleaner.make_dropdown_widget"><code>make_dropdown_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L126" class="source_link">[source]</a></h4>

> <code>make_dropdown_widget</code>(<b>`description`</b>=<b><i>`'Description'`</i></b>, <b>`options`</b>=<b><i>`['Label 1', 'Label 2']`</i></b>, <b>`value`</b>=<b><i>`'Label 1'`</i></b>, <b>`file_path`</b>=<b><i>`None`</i></b>, <b>`layout`</b>=<b><i>`Layout()`</i></b>, <b>`handler`</b>=<b><i>`None`</i></b>)

Return a Dropdown widget with specified `handler`.  